In [ ]:
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")
using PyCall
using Distributions
unshift!(PyVector(pyimport("sys")["path"]), "")
unshift!(PyVector(pyimport("sys")["path"]), "..")
unshift!(PyVector(pyimport("sys")["path"]), "../Ad05RTheta1RSteer05Abs0FR200")
unshift!(PyVector(pyimport("sys")["path"]), "../CommonFiles")
@pyimport python2juliaReuse as p2j

In [ ]:
# policies = ["Baseline/Rewards/b_15e3_3","Retrain/Rewards/rarl","Retrain/Rewards/rarlP10",
#             "FSP/Rewards/nBnoTR","FSP/Rewards/nBnoTRP10"]

In [ ]:
pc = p2j.PolicyContainer()
path = "Data/Sep1/Ad05RTheta1RSteer05Abs0FR200/Ad/"
# policies = readdir(string(path,"Policy"))
# pnames = []
# for i = 1:length(policies)
#     if policies[i] != ".DS_Store"
#         push!(pnames,policies[i][1:length(policies[i])-4])
#     end
# end
# println(pnames)
pnames = ["b_15e3","rarl_5e3","fsp_5e3"]
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")


In [ ]:
mutable struct Counter
    i::Int
    I::Int
    num_collision::Int
    rate_collision::Float64
    total_reward::Float64
    single_reward::Float64
    avg_reward::Float64
    max_reward::Float64
    min_reward::Float64
    rewards::Array{Float64}
    done::Bool
    direction::Int
    total_speed::Float64
    single_speed::Float64
    avg_speed::Float64
    speeds::Array{Float64}
    single_time::Float64
    total_time::Float64
    time_between_collisions::Float64
    total_reward_d::Float64
    single_reward_d::Float64
    avg_reward_d::Float64
    rewards_d::Array{Float64}
end
function Counter(N::Int)
    return Counter(0,N,0,0.0,0.0,0.0,0.0,-Inf,Inf,zeros(N),false,-1,
        0.0,0.0,0.0,zeros(N),0.0,0.0,0.0,0.0,0.0,0.0,zeros(N))
end

In [ ]:
function handle_action_ad!(pc,scene,models,roadway,counter,rarl)
    state = get_observation(scene,models,roadway)
    
    action1_arr, action2_arr = pc[:getAction](state)
    action1_arr = tanh.(action1_arr)
    action2_arr = tanh.(action2_arr)

    action = Egoaction(action1_arr[1],action1_arr[2],
                       action2_arr[1],action2_arr[2])
    
    done, bump = simulate_action!(action,scene,models,roadway)
    counter.single_time += TIMESTEP
    counter.total_time += TIMESTEP
    reward = reward_fn(action,done,scene,models,roadway)
    if bump
        counter.num_collision += 1
    else
        counter.total_reward_d += reward
        counter.single_reward_d += reward
    end
    
    counter.total_reward += reward
    counter.single_reward += reward
    
    speed = scene[1].state.v
    counter.single_speed += speed
    counter.total_speed += speed
    
    if done
        pc[:resetPolicy](1)
        rand_ego!(scene,models,roadway)
        
        counter.i += 1
        if counter.single_reward > counter.max_reward
            counter.max_reward = counter.single_reward
        end
        if counter.single_reward < counter.min_reward
            counter.min_reward = counter.single_reward
        end
        if counter.i > counter.I
            counter.done = true
            counter.avg_reward = counter.total_reward/counter.I
            counter.rate_collision = counter.num_collision/counter.I
            counter.avg_reward_d = counter.total_reward_d/counter.I
            counter.avg_speed = counter.total_speed/(counter.total_time/TIMESTEP)
            counter.time_between_collisions = counter.total_time/counter.num_collision
        else
            counter.rewards[counter.i] = counter.single_reward
            counter.rewards_d[counter.i] = counter.single_reward_d
            counter.speeds[counter.i] = counter.single_speed/(counter.single_time/TIMESTEP)
        end
        counter.single_reward = 0.0
        counter.single_reward_d = 0.0
        counter.single_speed = 0.0
        counter.single_time = 0.0
    end
    
    
end

In [ ]:
scene, models, roadway = initialize_env()
log_path = string(path,"Data/")
for pname in pnames
    println(pname)
#     files = readdir(string(path,"Policy/",pname))
#     itrs = []
#     for i = 1:length(files)
#         if files[i] != ".DS_Store"
#             push!(itrs,policies[i][1:length(files[i])-4])
#         end
#     end
    
    for alpha in 0:10:200
        println(alpha)
        policy_path = string(path,"Policy/",pname,"/itr_",alpha,".pkl")
        pc[:reset_policy](policy_path=policy_path)
        N = 200

        counter = Counter(N)
        rand_ego!(scene,models,roadway)
        pc[:resetPolicy](1)
        while !counter.done
            handle_action_ad!(pc,scene,models,roadway,counter,true)
        end
        println(counter.avg_reward)
        println(counter.rate_collision)

        writedlm(string(log_path,pname,"_rewards_ad",alpha,".txt"),counter.rewards)
        writedlm(string(log_path,pname,"_rewards_d_ad",alpha,".txt"),counter.rewards_d)
        writedlm(string(log_path,pname,"_speeds_ad",alpha,".txt"),counter.speeds)
        writedlm(string(log_path,pname,"_collision_rate_ad",alpha,".txt"),counter.rate_collision)
#         writedlm(string(log_path,pname,"_time_between_collisions_pareto",alpha,".txt"),(counter.total_time+counter2.total_time)/(counter.num_collision+counter2.num_collision))
    end
end